In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import pickle

In [105]:
initial_data = pd.read_csv('./data/algebra_2005_2006_train.txt', delimiter = "\t")
print(initial_data.shape)
initial_data.head()

(809694, 19)


,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,1,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,3(x+2) = 15,2005-09-09 12:24:35.0,2005-09-09 12:24:49.0,2005-09-09 12:25:15.0,2005-09-09 12:25:15.0,40.0,NaN,40.0,0,2,3,1,[SkillRule: Eliminate Parens; {CLT nested; CLT...,1
1,2,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG4-FIXED,1,x+2 = 5,2005-09-09 12:25:15.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,2005-09-09 12:25:31.0,16.0,16.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",1~~1
2,3,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,2-8y = -4,2005-09-09 12:25:36.0,2005-09-09 12:25:43.0,2005-09-09 12:26:12.0,2005-09-09 12:26:12.0,36.0,NaN,36.0,0,2,3,1,"[SkillRule: Remove constant; {ax+b=c, positive...",2
3,4,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,1,-8y = -6,2005-09-09 12:26:12.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,2005-09-09 12:26:34.0,22.0,22.0,NaN,1,0,0,1,"[SkillRule: Remove coefficient; {ax+b=c, divid...",1~~1
4,5,0BrbPbwCMz,"Unit ES_04, Section ES_04-1",EG40,2,-7y-5 = -4,2005-09-09 12:26:38.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,2005-09-09 12:28:36.0,118.0,118.0,NaN,1,0,0,1,"[SkillRule: Remove constant; {ax+b=c, positive...",3~~1


# Collecter les données

In [106]:
initial_data.columns

Index(['Row', 'Anon Student Id', 'Problem Hierarchy', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)'],
      dtype='object')

In [107]:
len(initial_data['Problem Hierarchy'].unique())

138

In [108]:
initial_data = initial_data.rename(columns={'Anon Student Id': 'user_id','Problem Name': 'item_id','Step Name': 'step_id','KC(Default)': 'kc_id','First Transaction Time': 'timestamp','Correct First Attempt': 'correct',"Hints":'nb_hint','Incorrects':'incorrect','Corrects':'incorrect_?'})
needed = initial_data[["user_id","item_id","timestamp","correct","nb_hint"]]
needed

,user_id,item_id,timestamp,correct,nb_hint
0,0BrbPbwCMz,EG4-FIXED,2005-09-09 12:24:49.0,0,3
1,0BrbPbwCMz,EG4-FIXED,2005-09-09 12:25:31.0,1,0
2,0BrbPbwCMz,EG40,2005-09-09 12:25:43.0,0,3
3,0BrbPbwCMz,EG40,2005-09-09 12:26:34.0,1,0
4,0BrbPbwCMz,EG40,2005-09-09 12:28:36.0,1,0
...,...,...,...,...,...
809689,IQQo3367x0,EG40,2006-03-09 10:58:13.0,1,0
809690,IQQo3367x0,EG40,2006-03-09 10:58:18.0,1,0
809691,IQQo3367x0,EG40,2006-03-09 10:58:22.0,0,3
809692,IQQo3367x0,EG40,2006-03-09 10:58:44.0,1,0


In [109]:
needed['timestamp'] = pd.to_datetime(needed['timestamp']).map(lambda t: t.timestamp()).round().astype(np.int32)

# Conservez uniquement les utilisateurs avec au moins 10 interactions
needed = needed.groupby("user_id").filter(lambda x: len(x) >= 1)

# Supprimer les entrées où le résultat n'est pas binaire mais continu
needed = needed[needed.correct.isin([0, 1])]
needed['correct'] = needed['correct'].astype(np.int32)

# Trier la colonne "timestamp" et changer le nom de la colonne
needed = needed.sort_values('timestamp')
print(needed.shape)
needed.head()

(809694, 5)


,user_id,item_id,timestamp,correct,nb_hint
535958,45TTYcotWk,LDEMO_WKST,1125395408,0,0
562642,HvghFVBS5v,LDEMO_WKST,1125395410,1,0
569598,IQ98biy227,LDEMO_WKST,1125395414,1,0
555691,Dmq6441349,LDEMO_WKST,1125395416,1,0
545705,80nlN05JQ6,LDEMO_WKST,1125395419,0,0


# Encodage

In [110]:
userId_encoder = LabelEncoder()
itemId_encoder = LabelEncoder()

needed["user_id"] = userId_encoder.fit_transform(needed["user_id"])
needed["item_id"] = itemId_encoder.fit_transform(needed["item_id"])

# Save user_id and item_id encoder
####
outputUserInEncoder = open('./final_data/userId_encoder.pkl', 'wb')
pickle.dump(userId_encoder, outputUserInEncoder)
outputUserInEncoder.close()
####
outputItemInEncoder = open('./final_data/itemId_encoder.pkl', 'wb')
pickle.dump(itemId_encoder, outputItemInEncoder)
outputItemInEncoder.close()

needed.head()

### Load saved encoder
"""
import pickle 
pkl_file = open('Departure_encoder.pkl', 'rb')
le_departure = pickle.load(pkl_file) 
pkl_file.close()
df_test['Departure'] = le_departure.transform(df_test['Departure'])
"""

"\nimport pickle \npkl_file = open('Departure_encoder.pkl', 'rb')\nle_departure = pickle.load(pkl_file) \npkl_file.close()\ndf_test['Departure'] = le_departure.transform(df_test['Departure'])\n"

In [111]:
# replace user_id and item_id where id == 0
needed["user_id"].replace(0,len(needed['user_id'].unique()),inplace=True)
needed["item_id"].replace(0,len(needed['item_id'].unique()),inplace=True)

In [112]:
## dataConfig 
users = len(needed['user_id'].unique())
items = len(needed['item_id'].unique())
n_observations = len(needed)
config = {
    'user_ids': users,
    'item_ids': items,
    'observations': n_observations,
}
config

{'user_ids': 574, 'item_ids': 1084, 'observations': 809694}

In [113]:
needed = needed.reset_index()
needed = needed.drop('index', 1)
needed

,user_id,item_id,timestamp,correct,nb_hint
0,72,563,1125395408,0,0
1,249,563,1125395410,1,0
2,251,563,1125395414,1,0
3,214,563,1125395416,1,0
4,155,563,1125395419,0,0
...,...,...,...,...,...
809689,395,482,1149678699,1,0
809690,395,482,1149678711,0,4
809691,395,482,1149678744,1,0
809692,395,482,1149678751,1,0


In [116]:
# Save needed Data to csv file
needed = needed[["user_id","item_id","correct","nb_hint"]]
needed.to_csv("./final_data/neededData.csv")

# Import irt predictions

In [115]:
# conditions = [
#     (needed['correct'] == 1) & (needed['nb_hint'] > 0),
#     (needed['correct'] == 1) & (needed['nb_hint'] == 0),
#     (needed['correct'] == 0) & (needed['nb_hint'] > 0),
#     (needed['correct'] == 0) & (needed['nb_hint'] == 0)
#     ]
# # 1 ==> Correct avec hint
# # 2 ==> Correct sans hint
# # 3 ==> Incorrect avec hint
# # 4 ==> Incorrect sans hint
# values = [1, 2, 3, 4]

# # create a new column and use np.select to assign values to it using our lists as arguments
# needed['answers_using_hint'] = np.select(conditions, values)
# needed.head()